In [1]:
import pandas as pd

import os

import torch
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.loader import DataLoader


from torch.nn import Linear, Parameter
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, MessagePassing
from torch_geometric.utils import add_self_loops, degree

### Preprocessing

In [ ]:
class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0], weights_only=False)

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        import os, torch
        import pandas as pd
        from torch_geometric.data import Data

        data_list = []
        root_dir = 'edkgdl_all_data'
        node_dir = 'Graph_index.txt'
        edge_direct_dir = 'Graph_edge_index_direct.txt'
        label_dir = 'Graph_label.txt'

        n = len([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        # n = 903

        b = pd.read_csv(os.path.join(root_dir, label_dir), header=None)

        for i in range(n):
            file_dir = str(i)
            node_path = os.path.join(root_dir, file_dir, node_dir)
            edge_path = os.path.join(root_dir, file_dir, edge_direct_dir)

            # nodes
            y = pd.read_csv(node_path, header=None)
            x = torch.tensor(y.values, dtype=torch.float)

            # edges
            a = pd.read_csv(edge_path, header=None)
            edge_index = torch.tensor(a.iloc[:, 0:2].T.values, dtype=torch.long)
            edge_attr = torch.tensor(a.iloc[:, 2:].values, dtype=torch.float)

            label = torch.tensor(b.iloc[i, 1], dtype=torch.long).reshape(1)

            data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=label)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [d for d in data_list if self.pre_filter(d)]
        if self.pre_transform is not None:
            data_list = [self.pre_transform(d) for d in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
dataset = MyOwnDataset("EDKG-DL_data")

print()
print(f'Dataset: {dataset}:')
print('==========================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0] # get the first graph object

print()
print(data)
print('==========================================================')

# gather some statistics about the first graph
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Processing...



Dataset: MyOwnDataset(903):
Number of graphs: 903
Number of features: 2
Number of classes: 2

Data(x=[74, 2], edge_index=[2, 657], edge_attr=[657, 2], y=[1])
Number of nodes: 74
Number of edges: 657
Average node degree: 8.88
Has isolated nodes: True
Has self-loops: False
Is undirected: False


Done!


In [ ]:
torch.manual_seed(12345)
dataset = dataset.shuffle()

train_dataset = dataset[:87] + dataset[87:903]
test_dataset = dataset[:87] + dataset[87:903]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graph: {len(test_dataset)}')

Number of training graphs: 903
Number of test graph: 903


In [5]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=========')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(x=[4736, 2], edge_index=[2, 42048], edge_attr=[42048, 2], y=[64], batch=[4736], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(x=[4736, 2], edge_index=[2, 42048], edge_attr=[42048, 2], y=[64], batch=[4736], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(x=[4736, 2], edge_index=[2, 42048], edge_attr=[42048, 2], y=[64], batch=[4736], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(x=[4736, 2], edge_index=[2, 42048], edge_attr=[42048, 2], y=[64], batch=[4736], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(x=[4736, 2], edge_index=[2, 42048], edge_attr=[42048, 2], y=[64], batch=[4736], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(x=[4736, 2], edge_index=[2, 42048], edge_attr=[42048, 2], y=[64], batch=[4736], ptr=[65])

Step 7:
Number of graphs in the current batch: 64
DataBatch(x=[4736, 2], edge_index=[2, 42048]

# Train

In [6]:
print(torch.cuda.device_count())

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def try_gpu(i=0):  #@save
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

2


In [ ]:
# === W&B ===
import os, sys, pathlib, subprocess, glob, time

PROJECT = "EDCs-BT-GNN-classification"
WANDB_DIR = r"E:\THY\EDC-AOP\GNN\wandb_runs"
pathlib.Path(WANDB_DIR).mkdir(parents=True, exist_ok=True)

os.environ.update({
    "WANDB_MODE": "offline",
    "WANDB_DIR": WANDB_DIR,
    "WANDB_DISABLE_SERVICE": "true",
    "WANDB_DISABLE_CODE": "true",
    "WANDB_DISABLE_GIT": "true",
    "WANDB_DISABLE_NOTEBOOK": "true",
    "WANDB_DISABLE_GPU": "true",
    "WANDB_HTTP_TIMEOUT": "30",
    "WANDB_INIT_TIMEOUT": "30",
})

import wandb

In [8]:
class GCNConvEdge(MessagePassing):
    def __init__(self, in_channels, out_channels, edge_channels):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.lin = Linear(in_channels, out_channels, bias=False)
        self.bias = Parameter(torch.empty(2 * out_channels))
        self.lin_edge = Linear(edge_channels, out_channels, bias=False)

        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()
        self.lin_edge.reset_parameters()

    def forward(self, x, edge_index, ex):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)
        ex = self.lin_edge(ex)
        extended_ex = torch.cat([ex, torch.zeros([x.shape[0],ex.shape[1]], device=ex.device, dtype=ex.dtype)], dim=0)

        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-5: Start propagating messages.
        out = self.propagate(edge_index, x=x, norm=norm, ex=extended_ex)

        # Step 6: Apply a final bias vector.
        out += self.bias

        return out, ex

    def message(self, x_j, norm, ex):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        return norm.view(-1, 1) * torch.cat([x_j, ex], dim=1)
    
# model 
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels_1, hidden_channels_2, hidden_channels_3):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConvEdge(dataset.num_node_features, hidden_channels_1, dataset.num_edge_features)
        self.conv2 = GCNConvEdge(2 * hidden_channels_1, hidden_channels_2, hidden_channels_1)
        self.conv3 = GCNConvEdge(2 * hidden_channels_2, hidden_channels_3, hidden_channels_2)
        self.lin = Linear(2 * hidden_channels_3, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_attr):
        # 1. obtain node embeddings and edge embeddings
        x, ex = self.conv1(x, edge_index, edge_attr)
        x = x.relu()
        x, ex = self.conv2(x, edge_index, ex)
        x = x.relu()
        x, _ = self.conv3(x, edge_index, ex)

        # 2. readout layer
        x = global_mean_pool(x, batch) # [batch_size, hidden_channels]

        # 3. apply a final classifier 
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x

In [9]:
import tqdm

def agent_fn():
    wandb.init()
    model = GCN(
       hidden_channels_1=wandb.config.hidden_size1,
       hidden_channels_2=wandb.config.hidden_size2,
       hidden_channels_3=wandb.config.hidden_size3
    ).to(device=try_gpu())
    wandb.watch(model)

    optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.lr, weight_decay=wandb.config.weight_decay)
    criterion = torch.nn.CrossEntropyLoss()

    batch_size = wandb.config.batch_size
    train_iter = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_iter = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    def train(loader):
      model.train()

      for data in loader: # iterate in batches over the training dataset.
        data = data.to(device=try_gpu())
        out = model(data.x, data.edge_index, data.batch, data.edge_attr) # perform a single forward pass.
        loss = criterion(out, data.y) # compute the loss
        loss.backward() # derive gradients.
        optimizer.step() # update parameters based on gradients.
        optimizer.zero_grad() # clear gradients.
        
    def test(loader):
      model.eval()
      
      TP = 0
      TN = 0
      FP = 0
      FN = 0
      for data in loader:
        data = data.to(device=try_gpu())
        labels = data.y.squeeze()
        out = model(data.x, data.edge_index, data.batch, data.edge_attr)
        pred = out.argmax(dim=1)
        for i in range(len(labels)):
          if pred[i] == 1 and labels[i] == 1: # TruePositive
            TP = TP + 1
          elif pred[i] == 0 and labels[i] == 0: # TrueNegative
            TN = TN + 1
          elif pred[i] == 1 and labels[i] == 0: # FalsePositive
            FP = FP + 1
          else: # FalseNegative
            FN = FN + 1
        acc = (TP + TN) / (FN + FP + TP + TN)
      return TP, TN, FP, FN, acc


    for epoch in tqdm.tqdm(range(1, 201)):
        train(train_iter)
        _, _, _, _, train_acc = test(train_loader)
        _, _, _, _, test_acc = test(test_loader)
        wandb.log({"GCNConvEdge-4Layers/train_acc": train_acc})
        wandb.log({"GCNConvEdge-4Layers/test_acc": test_acc})


    model.eval()
    TP, TN, FP, FN, acc = test(test_iter)
    if TP + FP > 0:
       pre = TP/ (TP + FP)
    else:
       pre = 0
    if TP + FN > 0:
       recall = TP / (TP + FN)
    else:
       recall = 0
    if pre + recall > 0:
       F1 = 2 * (pre * recall) / (pre + recall)
    else:
       F1 = 0

    wandb.summary["GCNConvEdge-4Layers/TruePositives"] = TP
    wandb.summary["GCNConvEdge-4Layers/TrueNegatives"] = TN
    wandb.summary["GCNConvEdge4Layers/FalsePositives"] = FP
    wandb.summary["GCNConvEdge-4Layers/FalseNegatives"] = FN
    wandb.summary["GCNConvEdge-4Layers/Accuracy"] = acc
    wandb.summary["GCNConvEdge-4Layers/Precision"] = pre
    wandb.summary["GCNConvEdge-4Layers/Recall"] = recall
    wandb.summary["GCNConvEdge-4Layers/F1-score"] = F1
    wandb.log({"GCNConvEdge-4Layers/TruePositives": TP})
    wandb.log({"GCNConvEdge-4Layers/TrueNegatives": TN})
    wandb.log({"GCNConvEdge-4Layers/FalsePositives": FP})
    wandb.log({"GCNConvEdge-4Layers/FalseNegatives": FN})
    wandb.log({"GCNConvEdge-4Layers/Accuracy": acc})
    wandb.log({"GCNConvEdge-4Layers/Precision": pre})
    wandb.log({"GCNConvEdge-4Layers/Recall": recall})
    wandb.log({"GCNConvEdge-4Layers/F1-score": F1})

    # 获取当前 run 的 files 目录
    run_dir = wandb.run.dir  # 形如 .../offline-run-xxxx/files

    # 保存模型 state_dict
    model_path = os.path.join(run_dir, "model_state.pt")
    torch.save(model.state_dict(), model_path)

    wandb.finish()

In [10]:
sweep_config = {
    "name": "GCNConvEdge-4Layers",
    "method": "bayes",
    "metric": {
        "name": "GCNConvEdge-4Layers/F1-score",
        "goal": "maximize",
    },
    "parameters": {
        "hidden_size1": {
            "values": [100, 80, 60, 50, 40, 30, 20, 10]
        },
        "hidden_size2": {
            "values": [100, 80, 60, 50, 40, 30, 20, 10]
        },
        "hidden_size3": {
            "values": [100, 80, 60, 50, 40, 30, 20, 10]
        },
        "weight_decay": {
            "distribution": "normal",
            "mu": 5e-4,
            "sigma": 1e-5,
        },
        "lr": {
            "min": 1e-4,
            "max": 1e-3
        },
        "batch_size": {
            "values": [300, 200, 100, 50, 20]
        }
    }
}

In [11]:
# Register the Sweep with W&B
sweep_id = wandb.sweep(sweep_config, project=PROJECT)

wandb: WARNING Unable to verify login in offline mode.


Create sweep with ID: hwetukn4
Sweep URL: https://wandb.ai/haoyuejudy/EDCs-BT-GNN-classification/sweeps/hwetukn4


In [12]:
# Run the Sweeps agent
wandb.agent(sweep_id, project=PROJECT, function=agent_fn, count=100)

wandb: WARNING Unable to verify login in offline mode.
wandb: Agent Starting Run: t4ufj57v with config:
wandb: 	batch_size: 50
wandb: 	hidden_size1: 60
wandb: 	hidden_size2: 10
wandb: 	hidden_size3: 40
wandb: 	lr: 0.0004747885725318372
wandb: 	weight_decay: 0.0004910197348067454


100%|██████████| 200/200 [04:25<00:00,  1.33s/it]


GCNConvEdge-4Layers/Accuracy,▁
GCNConvEdge-4Layers/F1-score,▁
GCNConvEdge-4Layers/FalseNegatives,▁
GCNConvEdge-4Layers/FalsePositives,▁
GCNConvEdge-4Layers/Precision,▁
GCNConvEdge-4Layers/Recall,▁
GCNConvEdge-4Layers/TrueNegatives,▁
GCNConvEdge-4Layers/TruePositives,▁
GCNConvEdge-4Layers/test_acc,▁▁▁▁▁▁▁▁▆▆▆▆▆▆▆▆▆█▆▆▆▆▆▆▆██▆▆▆█▆▆▆▆▆▆▆▇▆
GCNConvEdge-4Layers/train_acc,▁▁▁▁▁▆▆▇▇▇▇▇▇▇▇▆▇▇▇██▇▇▆█▆▇▇▇▆█▄▇▇█▇▇▇▆█
GCNConvEdge-4Layers/Accuracy,0.93577


wandb: Agent Starting Run: z57gdbrv with config:
wandb: 	batch_size: 200
wandb: 	hidden_size1: 20
wandb: 	hidden_size2: 80
wandb: 	hidden_size3: 40
wandb: 	lr: 0.0005855525593884546
wandb: 	weight_decay: 0.0005150926605720456


100%|██████████| 200/200 [03:35<00:00,  1.08s/it]


...

100%|██████████| 200/200 [05:00<00:00,  1.50s/it]


GCNConvEdge-4Layers/Accuracy,▁
GCNConvEdge-4Layers/F1-score,▁
GCNConvEdge-4Layers/FalseNegatives,▁
GCNConvEdge-4Layers/FalsePositives,▁
GCNConvEdge-4Layers/Precision,▁
GCNConvEdge-4Layers/Recall,▁
GCNConvEdge-4Layers/TrueNegatives,▁
GCNConvEdge-4Layers/TruePositives,▁
GCNConvEdge-4Layers/test_acc,▁▁▁▁▁▁▁▆▅▆▅▆▆▆█▆▆▆▆▆▆█▆▆▆▆▆▆▆█▆▆▆▆▅▆▆▆█▆
GCNConvEdge-4Layers/train_acc,▁▁▁▁▁▁▁▁▁▁▅▅▅▅▅▇▆▇▇▆▅▆█▆▇▆▆▆▆▆▆▆▆▇▆▆▇▆▆▆
GCNConvEdge-4Layers/Accuracy,0.93577


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: whzmyhto with config:
wandb: 	batch_size: 300
wandb: 	hidden_size1: 40
wandb: 	hidden_size2: 60
wandb: 	hidden_size3: 50
wandb: 	lr: 0.0008756647075625584
wandb: 	weight_decay: 0.0005074110087506592


100%|██████████| 200/200 [03:58<00:00,  1.19s/it]


GCNConvEdge-4Layers/Accuracy,▁
GCNConvEdge-4Layers/F1-score,▁
GCNConvEdge-4Layers/FalseNegatives,▁
GCNConvEdge-4Layers/FalsePositives,▁
GCNConvEdge-4Layers/Precision,▁
GCNConvEdge-4Layers/Recall,▁
GCNConvEdge-4Layers/TrueNegatives,▁
GCNConvEdge-4Layers/TruePositives,▁
GCNConvEdge-4Layers/test_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁█▁█
GCNConvEdge-4Layers/train_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██▃
GCNConvEdge-4Layers/Accuracy,0.9103


In [ ]:
import os, sys, subprocess, glob

def sync_path(p):
    env = os.environ.copy()
    env["WANDB_MODE"] = "online"
    print("Syncing:", p)
    if p.endswith(os.sep + "files"):
        p = os.path.dirname(p)
    if os.path.exists(p):
        subprocess.run([sys.executable, "-m", "wandb", "sync", p], check=False, env=env)

for p in glob.glob(r"E:\THY\EDC-AOP\GNN\wandb_runs\wandb\offline-run-*"):
    sync_path(p)

for p in glob.glob(r"E:\THY\EDC-AOP\GNN\wandb_runs\wandb\sweep-*"):
    sync_path(p)

print("All done. Check your project on W&B.")

Syncing: E:\THY\EDC-AOP\GNN\wandb_runs\wandb\offline-run-20250811_215727-t4ufj57v
Syncing: E:\THY\EDC-AOP\GNN\wandb_runs\wandb\offline-run-20250811_220201-z57gdbrv
Syncing: E:\THY\EDC-AOP\GNN\wandb_runs\wandb\offline-run-20250811_220558-bq48ollz
Syncing: E:\THY\EDC-AOP\GNN\wandb_runs\wandb\offline-run-20250811_220935-qvozpzxf
Syncing: E:\THY\EDC-AOP\GNN\wandb_runs\wandb\offline-run-20250811_221415-whzmyhto
Syncing: E:\THY\EDC-AOP\GNN\wandb_runs\wandb\offline-run-20250811_221758-xzeaeacm
Syncing: E:\THY\EDC-AOP\GNN\wandb_runs\wandb\offline-run-20250811_222153-azyt2ldt
Syncing: E:\THY\EDC-AOP\GNN\wandb_runs\wandb\offline-run-20250811_222605-t7m1ebtm
Syncing: E:\THY\EDC-AOP\GNN\wandb_runs\wandb\offline-run-20250811_223109-rru00m7x
Syncing: E:\THY\EDC-AOP\GNN\wandb_runs\wandb\offline-run-20250811_223651-sieu5a72
Syncing: E:\THY\EDC-AOP\GNN\wandb_runs\wandb\offline-run-20250811_224020-ok7e2zu2
Syncing: E:\THY\EDC-AOP\GNN\wandb_runs\wandb\offline-run-20250811_224440-q1991kpf
Syncing: E:\THY\